# TD2 : classification de données textuelles

Ce notebook a été développé dans le cours donné par J. Velcin et J. Cugliari sur le Deep Learning à l'Université de Lyon 2.

On commence par charger en mémoire les données spam diffusée à l'occasion du tutoriel de A. Gramfort et A. Mueller à SciPy 2017
https://github.com/amueller/scipy-2017-sklearn

In [3]:
import numpy as np

import os

with open(os.path.join("SMSSpamCollection.txt")) as f:
    lines = [line.strip().split("\t") for line in f.readlines()]

text = [x[1] for x in lines]
y = [int(x[0] == "spam") for x in lines]

In [4]:
print(text[0:5])
print(y[0:5])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though"]
[0, 0, 1, 0, 0]


La bibliothèque scikit-learn fournit des commandes très utiles pour vectoriser le texte, cf. tutoriel SciPy 2017 et cours de text mining.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize_spamdata = TfidfVectorizer()
vectorize_spamdata.fit(text)
data = vectorize_spamdata.transform(text).toarray()

In [15]:
#il va decouper le document en mots d'ou les 5574 doc contiennent 8716 mots différents
vectorize_spamdata.get_feature_names_out()

array(['00', '000', '000pes', ..., 'zoom', 'zouk', 'zyada'], dtype=object)

In [9]:
dim = data.shape[1]
print(data.shape)
data[10:20, 4:8]

(5574, 8716)


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

On sépare le jeu de données en ensemble d'entraînement et de test en conservant un équilibre dans les classes.

In [15]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(data, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123,
                                                   stratify=y)

On construit un simple MLP avec une couche cachée, cf. TD 1.

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def mlp():
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
simple_mlp = mlp()
simple_mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 69736     
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 69,745
Trainable params: 69,745
Non-trainable params: 0
_________________________________________________________________


On a besoin de convertir la liste en tableau numpy.

In [18]:
test_y = np.array(test_y)
train_y = np.array(train_y)

On lance l'apprentissage sur 10 epochs avec des batch de 10 textes.

In [19]:
simple_mlp.fit(train_X, train_y, epochs=10, batch_size=10)

Epoch 1/10
391/391 [==============================] - 3s 4ms/step - loss: 0.4086 - accuracy: 0.8688
Epoch 2/10
391/391 [==============================] - 2s 5ms/step - loss: 0.1679 - accuracy: 0.9431
Epoch 3/10
391/391 [==============================] - 2s 4ms/step - loss: 0.0849 - accuracy: 0.9787
Epoch 4/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0487 - accuracy: 0.9903
Epoch 5/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0305 - accuracy: 0.9941
Epoch 6/10
391/391 [==============================] - 1s 4ms/step - loss: 0.0206 - accuracy: 0.9956
Epoch 7/10
391/391 [==============================] - 2s 4ms/step - loss: 0.0147 - accuracy: 0.9982
Epoch 8/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0109 - accuracy: 0.9987
Epoch 9/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0081 - accuracy: 0.9990
Epoch 10/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0062 - accuracy: 0.9992

Voyons les résultats en généralisation.

In [20]:
score = simple_mlp.evaluate(test_X, test_y)
print("test score: ", score[0])
print("test accuracy: ", score[1])

53/53 [==============================] - 1s 3ms/step - loss: 0.0513 - accuracy: 0.9863
test score:  0.051328688859939575
test accuracy:  0.9862522482872009


Pour information, la régression logistique atteint ~96% de réussite sur le même jeu de données.